In [70]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re
import os
import random

In [61]:
%%time
# naivebayes 垃圾邮件分类器

# dataset：英文邮件
train_path = "/Users/gunanxi/PycharmProjects/JustForFun/LightGBM/build/LightGBM/python-package/LSTM_learn/MachineLearning/email/train1"
test_path = "/Users/gunanxi/PycharmProjects/JustForFun/LightGBM/build/LightGBM/python-package/LSTM_learn/MachineLearning/email/test1"

   
def load_data(folder_path):
    print("Loading dataset ...")
    datalist = datasets.load_files(folder_path)
    print("summary: {0} documents in {1} categories.".format(len(datalist.data),len(datalist.target_names)))
    return datalist
train = load_data(train_path)
test = load_data(test_path)

Loading dataset ...
summary: 36 documents in 2 categories.
Loading dataset ...
summary: 36 documents in 2 categories.
<class 'sklearn.utils.Bunch'>
CPU times: user 3.63 ms, sys: 3.61 ms, total: 7.24 ms
Wall time: 5.75 ms


In [72]:
# 文本处理，文本转向量
def dropStopwordsAndToken(word_bag,essaydata,word_sum):
    # word_bag:dict
    essaydata = essaydata.decode('utf-8')
    data = re.sub("[\s+\.\!\/_,$%^*(+\"\'-]+|[+——！，。？'、~@#￥%……&*（）<>]+", " ", essaydata)
    data = re.sub("\d", " ", essaydata)
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(data)

    word_list = [w for w in word_tokens if not w in stop_words]

    for w in word_list: 
        if len(w) < 2:
            continue
        if w not in stop_words:
            w = w.encode("utf-8") #全部转utf-8编码
            w.lower()             #全部转小写
            if w in word_bag:
                word_bag[w] = word_bag[w] + 1
            else:
                word_bag[w] = 1.0
            word_sum = word_sum + word_bag[w]
    return word_bag,word_sum

def data_todf(data,word_bag,word_sum):
    for s in data:
        word,num = {},0
        word,num =dropStopwordsAndToken(word,s,num)
        word_bag.append(word)
        word_sum.append(num)
    return word_bag,word_sum

word_bag,word_sum = [],[]
word_bag,word_sum = data_todf(train.data,word_bag,word_sum)
word_bag,word_sum = data_todf(test.data,word_bag,word_sum)

df = pd.DataFrame(word_bag)
df.fillna(0, inplace = True)
print(df.shape)

train_target = pd.DataFrame(train.target)
test_target = pd.DataFrame(test.target)
target = pd.concat([train_target,test_target],axis=0,sort=False)
print(target.shape)

X_train,X_test, y_train, y_test = train_test_split(df,target,test_size=0.3, random_state=0)

(72, 3413)
(72, 1)


In [76]:
#model = GaussianNB() #0.72
model = MultinomialNB()
model.fit(X_train, y_train)
def predict(X):
    return model.predict(X)
res = predict(X_test)
print(accuracy_score(res,y_test))

0.7272727272727273


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
